In [1]:
import pandas as pd
import os,re,gensim
from tqdm import tqdm
import numpy as np
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel
from tabulate import tabulate
from warnings import filterwarnings
filterwarnings('ignore')

def ClusterClean(x) : 
  if ',' in str(x) : 
    List = [re.sub('[^A-Za-z0-9,]+', '', i).split(',')  for i in x.split('),')]
    List = [[i[0],int(i[1])] for i in List]
    ThisList = []
    for l in range(0,len(List),2) : 
      if l < len(List)-1 : 
        ThisList.append((List[l],List[l+1]))

    return List
  else :
    return x



EmptySentences = {}
EmptySentencesIndex = 0
def ClustersCostFunctionAllWords(Clusters,ShowResults = True) : 
  global GetAllWords,SavedWords
  global EmptySentences,EmptySentencesIndex
  
  ClustersDiffs = []
  BetweenDiffs = []
  if ShowResults : print('Calculating Within Clusters Variance')
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    AllWords = Cl.split("'")[1::2]
    if len(AllWords) > 5 : 
      Diffs = []
      for i in range(len(AllWords)-1) : 
        for j in range(i+1,len(AllWords)) : 
          if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
            Diffs.append(float(WordDifference(AllWords[i],AllWords[j])))
          else : 
            Diffs.append(0)
      if len(Diffs) <= 1 : 
        EmptySentencesIndex+=1
        EmptySentences[EmptySentencesIndex] = ['AllWords_Within',Clusters]
      ClustersDiffs.append(np.mean(Diffs))
      
  ClusterWords = []
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
  if ShowResults : print('Calculating Between Clusters Variance')
  for i in range(len(ClusterWords)) : 
    TheseWords = ClusterWords[i]
    ExList = [j for j in range(len(ClusterWords)) if j != i]
    OtherWords = []
    for k in ExList : 
      OtherWords.extend(ClusterWords[k])
    
    for Word1 in TheseWords : 
      for Word2 in OtherWords : 
        if Word1 in SavedWords.keys() and Word2 in SavedWords.keys() : 
          BetweenDiffs.append(float(WordDifference(Word1,Word2)))
        else : 
          BetweenDiffs.append(0)
  F1 = np.mean(BetweenDiffs)
  F2 = np.mean(ClustersDiffs)

  if len(ClustersDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['AllWords_Within',Clusters]
  if len(BetweenDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['AllWords_Between',Clusters]
  if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
  return F1,F2     


def WordDifference(Word1,Word2) :
  global SavedWords
  Word1,Word2 = Word1.lower(),Word2.lower()
  if not  Word1 in SavedWords.keys() or not  Word2 in SavedWords.keys() : print('Word not found')
  ThisResult = np.linalg.norm(np.array(SavedWords[Word1])-np.array(SavedWords[Word2]), ord = 2) 
  return ThisResult     



def ClustersCostFunctionMean(Clusters,ShowResults = True) : 
  global GetAllWords,SavedWords
  ClustersDiffs = []
  BetweenDiffs = []
  if ShowResults : print('Calculating Within Clusters Variance')
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    AllWords = Cl.split("'")[1::2]
    if len(AllWords) > 5 : 
      Diffs = []
      for i in range(len(AllWords)-1) : 
        for j in range(i+1,len(AllWords)) : 
          if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
            Diffs.append(float(WordDifference(AllWords[i],AllWords[j])))
          else : 
            Diffs.append(0)
      ClustersDiffs.append(np.mean(Diffs))
  ClusterWords = []
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
  if ShowResults : print('Calculating Between Clusters Variance')

  TheseWordsValuesMean = []
  for i in range(len(ClusterWords)) : 
    TheseWords = ClusterWords[i]
    for EachWord in TheseWords : 
      EachWord = EachWord.lower()
      if not EachWord in SavedWords.keys() : 
        if EachWord in EnglishWE.keys()  : 
          SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
        else : 
          SavedWords[EachWord] = [0 for i in range(300)]
    TheseWordsValues = [SavedWords[i] for i in TheseWords]
    if len(TheseWordsValues) > 0 : 
      TheseWordsValuesMean.append([np.matrix(TheseWordsValues).T[i].mean() for i in range(300)])
  
  for a in range(len(TheseWordsValuesMean)) : 
    for b in range(a+1,len(TheseWordsValuesMean)) : 
      BetweenDiffs.append(sum([(float(g)-float(m))**2 for g,m in zip(TheseWordsValuesMean[a],TheseWordsValuesMean[b])]))

  F1 = np.mean(BetweenDiffs)
  F2 = np.mean(ClustersDiffs)
  global EmptySentences,EmptySentencesIndex
  if len(ClustersDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Mean_Within',Clusters]
  if len(BetweenDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Mean_Between',Clusters]
  if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
  return F1,F2  






def ClustersCostFunctionMin(Clusters,ShowResults = True) : 
  global GetAllWords,SavedWords
  ClustersDiffs = []
  BetweenDiffs = []
  if ShowResults : print('Calculating Within Clusters Variance')
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    AllWords = Cl.split("'")[1::2]
    if len(AllWords) > 5 : 
      Diffs = []
      for i in range(len(AllWords)-1) : 
        for j in range(i+1,len(AllWords)) : 
          if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
            Diffs.append(float(WordDifference(AllWords[i],AllWords[j])))
          else : 
            Diffs.append(0)
      ClustersDiffs.append(np.mean(Diffs))
  ClusterWords = []
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
  if ShowResults : print('Calculating Between Clusters Variance')
  AllTheseWords = []
  for i in range(len(ClusterWords)) : 
    TheseWords = ClusterWords[i]
    for EachWord in TheseWords : 
      EachWord = EachWord.lower()
      if not EachWord in SavedWords.keys() : 
        if EachWord in EnglishWE.keys()  : 
          SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
        else : 
          SavedWords[EachWord] = [0 for i in range(300)]
    TheseWordsValues = [SavedWords[i] for i in TheseWords]
    AllTheseWords.append(TheseWordsValues)
  for EachGroup in AllTheseWords : 
    for OtherGroup in [Group for Group in AllTheseWords if Group != EachGroup ] : 
      GroupsWordsDiff = []
      for WordA in  EachGroup : 
        for WordB in OtherGroup : 
          if WordA != WordB : 
            GroupsWordsDiff.append(sum([(float(g)-float(m))**2 for g,m in zip(WordA,WordB)]))
      if len(GroupsWordsDiff) > 0 : BetweenDiffs.append(min(GroupsWordsDiff))

  F1 = np.mean(BetweenDiffs)
  F2 = np.mean(ClustersDiffs)
  global EmptySentences,EmptySentencesIndex
  if len(ClustersDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Min_Within',Clusters]
  if len(BetweenDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Min_Between',Clusters]
  if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
  return F1,F2    




def ClustersCostFunctionMax(Clusters,ShowResults = True) : 
  global GetAllWords,SavedWords
  ClustersDiffs = []
  BetweenDiffs = []
  if ShowResults : print('Calculating Within Clusters Variance')
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    AllWords = Cl.split("'")[1::2]
    if len(AllWords) > 5 : 
      Diffs = []
      for i in range(len(AllWords)-1) : 
        for j in range(i+1,len(AllWords)) : 
          if AllWords[i] in SavedWords.keys() and AllWords[j] in SavedWords.keys() : 
            Diffs.append(float(WordDifference(AllWords[i],AllWords[j])))
          else : 
            Diffs.append(0)
      ClustersDiffs.append(np.mean(Diffs))
  ClusterWords = []
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
  if ShowResults : print('Calculating Between Clusters Variance')

  AllTheseWords = []
  for i in range(len(ClusterWords)) : 
    TheseWords = ClusterWords[i]
    for EachWord in TheseWords : 
      EachWord = EachWord.lower()
      if not EachWord in SavedWords.keys() : 
        if EachWord in EnglishWE.keys()  : 
          SavedWords[EachWord] = [i for i in EnglishWE[EachWord]]
        else : 
          SavedWords[EachWord] = [0 for i in range(300)]
    TheseWordsValues = [SavedWords[i] for i in TheseWords]
    AllTheseWords.append(TheseWordsValues)
  for EachGroup in AllTheseWords : 
    for OtherGroup in [Group for Group in AllTheseWords if Group != EachGroup ] : 
      GroupsWordsDiff = []
      for WordA in  EachGroup : 
        for WordB in OtherGroup : 
          if WordA != WordB : 
            GroupsWordsDiff.append(sum([(float(g)-float(m))**2 for g,m in zip(WordA,WordB)]))
      
      if len(GroupsWordsDiff) > 0 : BetweenDiffs.append(max(GroupsWordsDiff))
  F1 = np.mean(BetweenDiffs)
  F2 = np.mean(ClustersDiffs)
  global EmptySentences,EmptySentencesIndex
  if len(ClustersDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Max_Within',Clusters]
  if len(BetweenDiffs) <= 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Max_Between',Clusters]
  if ShowResults : print(f'Between Clusters Variance is {F1} and Within Clusters Variance is {F2}')
  return F1,F2    

def ClustersCostFunctionCoherence(Clusters,ShowResults = True) : 
  ClusterWords = []
  for Cl in Clusters['Top Words'].split('Cluster')[1:] : 
    if len(Cl.split("'")[1::2]) > 5  : ClusterWords.append(Cl.split("'")[1::2])    
  if ShowResults : print('Calculating Between Clusters Variance')
  AllTheseDict = {}
  for i in range(len(ClusterWords)) : 
    TheseWords = ClusterWords[i]
    TheseWords = [k.lower().strip() for k in TheseWords]
    AllTheseDict[i] = TheseWords
  AllCoherence = []
  for a in range(len(list(AllTheseDict.keys()))) : 
    for b in range(a+1,len(list(AllTheseDict.keys()))) : 
      if len(AllTheseDict[a]) > 1 and len(AllTheseDict[b])> 1 : 
        topics = [AllTheseDict[a],AllTheseDict[b]]
        AllNeededWords = []
        AllNeededWords.extend(AllTheseDict[a])
        AllNeededWords.extend(AllTheseDict[b])
        gensim_dictionary = corpora.Dictionary()
        for i,w in enumerate(AllNeededWords) : 
          common_dictionary.token2id[w] = i
          gensim_dictionary.token2id[w] = i
        common_corpus = [gensim_dictionary.doc2bow(text) for text in topics]
        cm = CoherenceModel(topics=topics, corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass') 
        coherence = cm.get_coherence() 
        AllCoherence.append(float(coherence))

  global EmptySentences,EmptySentencesIndex
  if len(AllCoherence) < 1 : 
    EmptySentencesIndex+=1
    EmptySentences[EmptySentencesIndex] = ['Coherence',Clusters]

  if len(AllCoherence) > 0 : 
    return np.mean(AllCoherence)
  else : 
    return 0.0



ThisPath  =  r'/content/drive/MyDrive/Data/PhD/Week 24 All Data/Functions'
GetAllWords = []


ThisFile = os.path.join(ThisPath,'Clusters.csv')


ResultsData = pd.read_csv(ThisFile)  
AllWordsWEDF = pd.read_csv(os.path.join(ThisPath,'AllWordsWE.csv'))
EnglishWE = {k:v for k,v in zip(AllWordsWEDF['Word'].tolist(),[AllWordsWEDF.loc[j,:].tolist()[2:] for j in range(AllWordsWEDF.shape[0])])}
len(EnglishWE.keys())

NumberOfClusrers = len(ResultsData['Doc Number'][0].strip('[]').split())

ResultsData['Parameters'] = ResultsData['Parameters'].apply(lambda x : [float(i) for i in x.strip('[]').split(',')])
ResultsData['Doc Number'] = ResultsData['Doc Number'].apply(lambda x : [int(i) for i in x.strip('[]').split()])
ResultsData['Top Index'] = ResultsData['Top Index'].apply(lambda x : [int(i) for i in x.strip('[]').split()])
Losses = ResultsData['Loss'].tolist()
Losses.sort()
for i in range(NumberOfClusrers) : 
  ResultsData[f'Cluster {i}'] = ResultsData[f'Cluster {i}'].apply(lambda x :  ClusterClean(x))

for i in ResultsData['Top Words'].tolist() : 
  for j in i.split('Cluster')[1:] : 
    GetAllWords.extend(j.split("'")[1::2])


p = 0
for i in set(GetAllWords) : 
  if i in EnglishWE.keys() : 
    p+=1
print(f'Number of Exist Words is {p} and Percentage of Words Exists in English Words {100*np.round(p/len(set(GetAllWords)),3)} %')
SavedWords = {}
for word in GetAllWords : 
  if word in EnglishWE.keys() : 
    SavedWords[word] = [i for i in EnglishWE[word]]
print(f'Number of Words in SavedWords {len(list(SavedWords.keys()))} ')



Losses = ResultsData['Loss'].tolist()
Losses.sort()

SortedResults = []
for Loss in Losses : 
  for i in range(ResultsData.shape[0]) :
    if Loss ==ResultsData['Loss'][i] : 
      SortedResults.append({k:v for k,v in zip(list(ResultsData.keys()),[ResultsData[j][i] for j in list(ResultsData.keys())])})

for i in tqdm (range(len(SortedResults))) : 
  for Cl in SortedResults[i]['Top Words'].split('Cluster')[1:] : 
    AllWords = Cl.split("'")[1::2]

CostFunctionResults = {}
for i in tqdm (range(len(SortedResults))) : 
  ThisClusterValue = list(ClustersCostFunctionAllWords(SortedResults[i],ShowResults=False))
  CostFunctionResults[i] = {'AllBCV':ThisClusterValue[0],'AllWCV':ThisClusterValue[1]}

for i in tqdm (range(len(SortedResults))) : 
  ThisClusterValue = list(ClustersCostFunctionMean(SortedResults[i],ShowResults=False))
  CostFunctionResults[i]['MeanBCV'] = ThisClusterValue[0]
  CostFunctionResults[i]['MeanWCV'] = ThisClusterValue[1]

for i in tqdm (range(len(SortedResults))) : 
  ThisClusterValue = list(ClustersCostFunctionMin(SortedResults[i],ShowResults=False))
  CostFunctionResults[i]['MinBCV'] = ThisClusterValue[0]
  CostFunctionResults[i]['MinWCV'] = ThisClusterValue[1]

for i in tqdm (range(len(SortedResults))) : 
  ThisClusterValue = list(ClustersCostFunctionMax(SortedResults[i],ShowResults=False))
  CostFunctionResults[i]['MaxBCV'] = ThisClusterValue[0]
  CostFunctionResults[i]['MaxWCV'] = ThisClusterValue[1]
  CostFunctionResults[i]['CohBCV'] = ClustersCostFunctionCoherence(SortedResults[i],ShowResults=False)


for i in tqdm(CostFunctionResults.keys()): 
  AllBCV =  CostFunctionResults[i]['AllBCV']
  AllWCV =  CostFunctionResults[i]['AllWCV']
  MeanBCV =  CostFunctionResults[i]['MeanBCV']
  MinBCV =  CostFunctionResults[i]['MinBCV']
  MaxBCV =  CostFunctionResults[i]['MaxBCV']    
  CohBCV = CostFunctionResults[i]['CohBCV']                 

  ThisAllD,ThisMeanD,ThisMinD,ThisMaxD,ThisCohD = {},{},{},{},{}
  l=0
  for a in np.linspace(0.01, 1, num=5) : 
    for b in np.linspace(0.01, 1, num=5): 
      l+=1
      ThisAllD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(AllBCV))]
      ThisMeanD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MeanBCV))]
      ThisMinD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MinBCV))]
      ThisMaxD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(MaxBCV))]
      ThisCohD[l] = [a,b,float(a*float(AllWCV)) - float(b*float(CohBCV))]
  CostFunctionResults[i]['AllCF'] = ThisAllD
  CostFunctionResults[i]['MeanCF'] = ThisMeanD
  CostFunctionResults[i]['MinCF'] = ThisMinD
  CostFunctionResults[i]['MaxCF'] = ThisMaxD
  CostFunctionResults[i]['CohCF'] = ThisCohD
for i in range(len(SortedResults)) : 
  for Item in ['AllBCV','AllWCV','MeanBCV','MinWCV','MaxBCV','MeanWCV','MinBCV','MaxWCV','CohBCV','AllCF','MeanCF','MinCF','MaxCF','CohCF'] : 
    SortedResults[i][Item] = CostFunctionResults[i][Item]

Alpha,Beta,A,B,AllBCV,MeanBCV,MinBCV,MaxBCV,CohBCV,WCV,IDs,AllCF,MeanCF,MinCF,MaxCF,CohCF = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
n=0
for i in SortedResults : 
  for k in i['MeanCF'].values() : 
    MeanCF.append(k[2])
  for k in i['MinCF'].values() : 
    MinCF.append(k[2])
  for k in i['MaxCF'].values() : 
    MaxCF.append(k[2])      
  for k in i['CohCF'].values() : 
    CohCF.append(k[2])      
  for j,k in i['AllCF'].items() : 
    n+=1
    IDs.append(n)
    AllCF.append(k[2])
    AllBCV.append(i['AllBCV'])
    MeanBCV.append(i['MeanBCV'])
    MinBCV.append(i['MinBCV'])
    MaxBCV.append(i['MaxBCV'])
    CohBCV.append(i['CohBCV'])
    WCV.append(i['AllWCV'])
    A.append(k[0])
    B.append(k[1])
    Alpha.append(i['Parameters'][0])
    Beta.append(i['Parameters'][1])

ResultsDF = pd.DataFrame(list(zip(IDs,Alpha,Beta,A,B,WCV,AllBCV,AllCF,MeanBCV,MeanCF,MinBCV,MinCF,MaxBCV,MaxCF,CohBCV,CohCF)),columns = ['IDs','Alpha','Beta','A','B','WCV','AllBCV','AllCF','MeanBCV','MeanCF','MinBCV','MinCF','MaxBCV','MaxCF','CohBCV','CohCF'])
ResultsDF.to_csv(os.path.join(ThisPath ,'ResultsDF.csv'))



ThisFile = os.path.join(ThisPath,'ResultsDF.csv')

ResultsDF = pd.read_csv(ThisFile) 
if 'index' in ResultsDF.columns : ResultsDF.drop(['index'],axis=1,inplace=True)
if 'Unnamed: 0' in ResultsDF.columns : ResultsDF.drop(['Unnamed: 0'],axis=1,inplace=True)
CalculatedCFs = ['AllCF','MeanCF','MinCF','MaxCF','CohCF']
CFsDataFrame = pd.DataFrame()
for CF in CalculatedCFs : 
  ResultLists = []
  for A in ResultsDF['A'].unique() : 
    for B in ResultsDF['B'].unique() : 
      MinCF = ResultsDF[ResultsDF['A'] == A][ResultsDF['B'] == B][CF].min()
      ThisList = ResultsDF[ResultsDF[CF] == MinCF].iloc[0,:].tolist()
      ResultLists.append([ThisList[0]] + ThisList[3:5]+ThisList[1:3]+ThisList[5:])
  col_names = ['Trial Number','A','B','Alpha','Beta','WCV','AllBCV','AllCF','MeanBCV','MeanCF','MinBCV','MinCF','MaxBCV','MaxCF','CohBCV','CohCF']
  
  ThisCFsDataFrame = pd.DataFrame(ResultLists, columns=col_names)
  ThisCFsDataFrame['SelectedCF'] = CF
  print(tabulate(ResultLists, headers=col_names, tablefmt="fancy_grid"))
  CFsDataFrame = pd.concat([CFsDataFrame,ThisCFsDataFrame],axis=0)
CFsDataFrame.to_csv(os.path.join(ThisPath,'BestCFValue.csv'))        



ThisFile = os.path.join(ThisPath,'ResultsDF.csv')
if os.path.exists(ThisFile) : 
  data = pd.read_csv(ThisFile)  
  for alpha in data['Alpha'].unique() : 
    for beta in data['Beta'].unique() : 
      if len(data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()) !=1 : print('error')
      if len(data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()) !=1 : print('error')
      if len(data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()) !=1 : print('error')
      if len(data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()) !=1 : print('error')
      if len(data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()) !=1 : print('error')

  AllBCVDict = {}
  n=0
  for alpha in data['Alpha'].unique() : 
    for beta in data['Beta'].unique() : 
      n+=1
      AllBCVDict[n] = {'Alpha': alpha,
                      'Beta':beta,
                      'AllBCV':data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()[0],
                      'MeanBCV':data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()[0],
                      'MinBCV':data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()[0],
                      'MaxBCV':data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()[0],
                      'WBCV':data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0],
                      'CohBCV':data[data['Alpha']==alpha][data['Beta']==beta]['CohBCV'].unique()[0],
                      'EqualParameters_AllBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['AllBCV'].unique()[0],
                      'EqualParameters_MeanBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MeanBCV'].unique()[0],
                      'EqualParameters_MinBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MinBCV'].unique()[0],
                      'EqualParameters_MaxBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['MaxBCV'].unique()[0],
                      'EqualParameters_CohBCV' : data[data['Alpha']==alpha][data['Beta']==beta]['WCV'].unique()[0] -data[data['Alpha']==alpha][data['Beta']==beta]['CohBCV'].unique()[0]}
  data = pd.DataFrame(AllBCVDict).T
  data.fillna(99,inplace=True)
  data['MincF'] = data.apply(lambda x : min(x.AllBCV,x.MeanBCV,x.MinBCV,x.MaxBCV,x.WBCV,x.CohBCV,x.EqualParameters_AllBCV,x.EqualParameters_MeanBCV,x.EqualParameters_MinBCV,x.EqualParameters_MaxBCV,x.EqualParameters_CohBCV),axis=1)
  data = data.sort_values(['Alpha', 'Beta'],ascending = [True, True])

  data.to_csv(os.path.join(ThisPath,'CombinedCFs.csv'))

Number of Exist Words is 131 and Percentage of Words Exists in English Words 98.5 %
Number of Words in SavedWords 131 


100%|██████████| 12506/12506 [00:08<00:00, 1514.08it/s]


╒════════════════╤════════╤════════╤═════════╤════════╤═════════╤══════════╤══════════════╤═══════════╤════════════╤══════════╤═════════════╤══════════╤════════════╤══════════════╤═══════════╕
│   Trial Number │      A │      B │   Alpha │   Beta │     WCV │   AllBCV │        AllCF │   MeanBCV │     MeanCF │   MinBCV │       MinCF │   MaxBCV │      MaxCF │       CohBCV │     CohCF │
╞════════════════╪════════╪════════╪═════════╪════════╪═════════╪══════════╪══════════════╪═══════════╪════════════╪══════════╪═════════════╪══════════╪════════════╪══════════════╪═══════════╡
│          45326 │ 0.01   │ 0.01   │    0.2  │    1   │ 1.13413 │  1.1994  │ -0.000652707 │  0.383331 │  0.007508  │ 0.371185 │  0.00762946 │  8.10163 │ -0.069675  │  1.99996e-12 │ 0.0113413 │
├────────────────┼────────┼────────┼─────────┼────────┼─────────┼──────────┼──────────────┼───────────┼────────────┼──────────┼─────────────┼──────────┼────────────┼──────────────┼───────────┤
│          29277 │ 0.01   │ 0.2575 